# Disclaimer

These are tools developed to help me determine when my intellectual property is posted without my permission online. 
These are tools created to help me ensure that content uploaded by students to the site do not infringe my intellectual property rights.

They simply automate the process of searching the site and thus make it easier to determine which items infringe rights and which do not.

They are extremely rough and probably not very user friendly. That said, I am making them public in case others find them useful.  

These tools are provided in the hope that they will be useful to those seeking to protect their intellectual property, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

The tools are provided here with no warranty or claim that their use is consistent with any law, policy, terms of use policy, or any other constraint. IF YOU CHOOSE TO USE THEM, YOU AGREE TO REVIEW AND COMPLY WITH ALL RELEVANT POLICIES, INCLUDING SITE TERMS OF SERVICE, AND YOU AGREE THAT YOUR USE IS SOLELY AT YOUR OWN RISK.

Questions, requests, complaints, or other feedback (or pull-requests!) are welcome through this project's github page
        
        https://github.com/HatakoHaterson/CourseZero

Please understand that this is not an active project for me; I will monitor the project on github very infrequently.

-- 復讐者


View a static image of the main notebook (does not run) on Jupyter: 
    
    https://nbviewer.jupyter.org/github/HatakoHaterson/CourseZero/blob/master/Notebooks/Gather%20information.ipynb?flush_cache=true

Or run it on your own mini-server using Binder

    https://mybinder.org/v2/gh/HatakoHaterson/CourseZero/master?filepath=Notebooks%2FGather%20information.ipynb


# Get started

Once the notebook has opened in binder, a toolbar will appear at the top of the notebook containing several buttons. The 'run' button is the most important.

After you've opened the notebook in binder, click the run button above a few times. Each click runs the code in the highlighted cell. You can also press shift+return to run it. You can change which cell is highlighted with the up and down arrows or by just clicking on it.

Try it with the next cell:

In [ ]:
print("You've successfully run this cell!")

Since these are rough-and-ready tools, I've left in much of the code. If you want to follow along, I've left comments to myself which identify the various steps. These are either single lines which start with a '#', like this: 
    
    # Query the server for records
    code code code
    
Or, multiple lines enclosed with three double quotes: 
    
    """blah blah
    tacos are good
    """  

You'll be able to tell that a cell is running by an asterisk appearing in the brackets to the left ( [ * ]). Some tasks may take a couple of minutes to run, so it may appear frozen.

I haven't set it up with an easy way to reset things. If you want to search a different campus or different set of departments, use the 'restart kernel' button above (to the right of the run button).

The next cell will grab and import all the code that operates behind the scenes. (If you're curious, you can check it out here: https://github.com/HatakoHaterson/CourseZero)

In [1]:
# for binder
# !pip install CourseZero

# for local testing
%cd ../

import json
import pandas as pd

import requests
from bs4 import BeautifulSoup

import environment as env
from CourseZero.FileSystemTools import getSystemRoot

from IPython.display import display, clear_output
from ipywidgets import widgets

from CourseZero.RequestTools import get_docs_for_campuses, get_docs_for_campus
# from CourseZero.Store import DataStore
from CourseZero.Widgets.CampusAndProfIdentifiers import make_campus_selector
from CourseZero.FileSystemTools import getSystemRoot
from CourseZero.DataStorageTools import load_campus_id_data

from CourseZero.Errors import UnsetValue

from CourseZero.DataHandlingTools import parse_json_into_df,get_departments, filter_by_dept_abbrevs, get_by_course_id
from CourseZero.Widgets.Inputs import dept_selection
from CourseZero.Widgets.Inputs import make_course_b
from CourseZero.Widgets.Inputs import make_link, make_infringement_b, make_html_url_list
from CourseZero.Store import TakedownStore

from CourseZero.Widgets.TakedownInputs import make_text_input, make_email_helper_button
import datetime


from environment import *


/Users/adam/Dropbox/CourseZero


In [ ]:
# DataStore.data.to_csv("{}/testdata.csv".format(env.PROJ_BASE))
# DataStore.data = pd.read_csv("{}/testdata.csv".format(env.PROJ_BASE))

# Select your campus


Running the next cell will display a selection menu. Use it to select the CSU you want to search.

NB, for now this will only work with CSU campuses

In [2]:
def show_email_draft_area():
    field_map = {r['prop'] : make_text_input(r) for r in TakedownStore.input_fields }
    inputs = list(field_map.values())

    def make_email_draft(**kwargs):
        letter_date = datetime.date.isoformat( datetime.date.today() )
        doc_urls = make_html_url_list(TakedownStore.data_store.infringing_docs)

        with open('{}/takedown_request.html'.format(env.TEMPLATE_FOLDER), 'r') as o:
            template = o.read()

        formatted = template.format(letter_date=letter_date, doc_urls=doc_urls, **kwargs)
        return display(widgets.HTML(value=formatted))

    # Show the email draft and fields to fill it in
    email_out = widgets.interactive_output(make_email_draft, field_map )
    return display(widgets.VBox([widgets.VBox(inputs), email_out]))
# show_email_draft_area()

In [3]:

def search_depts(data_store):
    # Query for results and parse them into a dataframe
    try:
        
        # dev !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
        json_data = get_docs_for_campus(data_store.campus_id, data_store.campus_name)
        data_store.data = parse_json_into_df(json_data)
        
        
        print("{} courses identified at {}".format(len(data_store.data), data_store.campus_name))
        print("{} department names located".format(len(data_store.departments)))
        
        # Create the output areas
        progress_out = widgets.Output(layout={'border': '1px solid black'})
        display(progress_out)
        depts_out = widgets.Output(layout={'border': '1px solid black'})
        display(depts_out)
        courses_out = widgets.Output(layout={'border': '1px solid black'})    
        display(courses_out)
        docs_out = widgets.Output(layout={'border': '1px solid black'})
        display(docs_out)
        takedown_out = widgets.Output(layout={'border': '1px solid black'})
        display(takedown_out)
        email_out = widgets.Output(layout={'border': '1px solid black'})
        display(email_out)
        
        TakedownStore.data_store = data_store


        def search_courses(data_store):
            # Make buttons for courses corresponding to the selected departments
            course_buttons =[]
            for i, r in data_store.selected_departments_data.iterrows():
                course_buttons.append(make_course_b(r, data_store, search_documents))

            buttons = widgets.VBox(course_buttons)
            label = widgets.HTML(value="<h1>Select courses to search</h1>")
            courses_out.clear_output()
            with courses_out:
                display(widgets.VBox([label, buttons]))

        def search_documents(data_store):
            """Displays selection buttons for the documents associated with the selected courses"""
            rows = [widgets.HBox([make_infringement_b(d, data_store, show_takedown), widgets.HTML(value=make_link(d))]) for d in data_store.selected_courses_documents_urls]
            docs_out.clear_output()
            with open('{}/file_selection_instructions.html'.format(env.TEMPLATE_FOLDER), 'r') as o:
                doc_instructions = o.read()

            doc_instructions = widgets.HTML(value=doc_instructions)
            with docs_out:
                display(widgets.VBox([doc_instructions, widgets.VBox(rows)]))

        def show_takedown(data_store):
            """Callback which controls whether the information about submitting a takedown request 
            is displayed"""
            if len(data_store.infringing_docs) == 0: pass
            
            with open('{}/takedown_instructions.html'.format(env.TEMPLATE_FOLDER), 'r') as o:
                takedown_instructions = o.read()
                # add list of urls
                url_list = make_html_url_list(data_store.infringing_docs)
                format_args = {'url_list':url_list}
                takedown_instructions = takedown_instructions.format(**format_args)
                takedown_instructions = widgets.HTML(value=takedown_instructions)
            
            takedown_out.clear_output()
            
            with takedown_out:
                display(widgets.VBox([takedown_instructions, make_email_helper_button(show_email_draft_area)]))


        # Show the department selection
        # buttons and other output areas
        with depts_out:
            dept_selection(data_store, search_courses)
    
    except UnsetValue as e:
        print(e.message)

# dev !!!!!!!!!!!!!!!!!!
# search_depts(DataStore)




In [4]:
# todo: this needs a default non csu campus

make_campus_selector(DataStore, search_depts)

Dropdown(description='Campus', layout=Layout(width='80%'), options=('California State University Bakersfield',…

Searching for California Maritime Academy
34 courses identified at California Maritime Academy
23 department names located


Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

Output(layout=Layout(border='1px solid black'))

In [ ]:
out = widgets.Output(layout={'border': '1px solid black'})
out2 = widgets.Output(layout={'border': '1px solid black'})


In [ ]:
out

In [ ]:
out2

In [ ]:
with out:
    display(widgets.IntSlider())

In [ ]:
DataStore.selected_courses_documents

In [ ]:
def progress_bar():
    return widgets.FloatProgress(
    value=7.5,
    min=0,
    max=10.0,
    step=0.1,
    description='Loading:',
    bar_style='info',
    orientation='horizontal'
)

In [ ]:
DataStore.campus_id

In [ ]:
DataStore.campus_name

In [ ]:
# add check to make sure they actually selected something
def check_selected(data_store):
    if data_store.campus_id is None:
#         out = widgets.Output(layout={'border': '5px solid red'})
#         out.append_display_data("Please select a campus to search using the dropdown menu above")
        print("Please select a campus to search using the dropdown menu above")
check_selected(DataStore)

# Search for courses on the selected campus

The next couple of cells will actually search for courses at the selected campus.

NB, when the search starts it will display a message like "Searching for California State University Dominguez hills". You may need to wait a few seconds for it to complete.

When the search completes, it will display buttons with all the departments found for the campus. Select all the departments you want to search for materials from..

The site does not seem to standardize the departments students enter. Thus your class may be identified in some rather odd ways.



# Select department names / abbreviations

The site does not seem to standardize the departments students enter. Thus your class may be identified in some rather odd ways.

Run the next cell and select all the possible departments using the buttons which display.

# Display all courses with uploaded materials for the selected departments

Run the next cell and select any of the courses which look like they may be yours

# Find material uploaded for selected courses

The following cells will query the site and retrieve the url's for all files that students have uploaded for the courses you've selected.

You will need the file url's to request that the site remove the content via this website: https://www.coursehero.com/copyright-infringement/

Remember that the url's this tool returns are simply all the files uploaded for the selected courses. 

*You must review the url's to determine that they infringe your rights.* 

Students may have uploaded their own intellectual property related to your course. You will infringe their rights if you request that their intellectual property be removed.

In [ ]:
from CourseZero.RequestTools import get_file_links_from_course_page
from CourseZero.Widgets.Inputs import show_selected_urls
from CourseZero.Store import TakedownStore
from CourseZero.Widgets.Inputs import make_link, make_infringement_b


# Testing
docs = ['url1', 'url2', 'url3']

# # For reals
# # docs = get_urls(data)
# # docs = DataStore.selected_courses_documents_urls

# # show_selected_urls(data)
# def search_documents(documents):
#     """Displays selection buttons for the documents associated with the selected courses"""
#     rows = [widgets.HBox([make_infringement_b(d, TakedownStore), widgets.HTML(value=make_link(d))]) for d in documents]
#     display(widgets.VBox(rows))

# Request takedown

CourseHero requests that you use their online form for submitting takedown requests. The form is https://www.coursehero.com/copyright-infringement/

However, they also provide the alternative of emailing or faxing a letter to DMCA@coursehero.com

You can generate the text of an email below.

    To file a DMCA takedown request, please complete the Copyright Infringement Notification form below.

    You will need to identify the infringing work(s) by providing the specific identification and location of the infringing work(s), including its URL or URLs on Course Hero. Each URL should be in the following format: https://www.coursehero.com/file/xxxx/...

    Once you submit a complete takedown request using our Copyright Infringement Notification form, our DMCA Compliance Team will quickly take action to remove the infringing content. You will be notified once the content has been removed.

    We are dedicated to helping copyright holders protect their work. As an an alternative to completing this form, you can email or fax your DMCA notification to DMCA@coursehero.com. If you have any questions or need assistance in completing your takedown request, please contact us at DMCA@coursehero.com.


Review each document below by clicking on the url (it should open the document on the CourseHero website in a new window/tab). If you believe it infringes your copyright, click the corresponding button on the right.

## Help with submitting through the webform

If you would like to submit the request using CourseHero's webform, here are the document names (as submitted by the student) and the corresponding url's

In [ ]:
# display non-link formatted urls and doc names

In [ ]:
# todo: change to use the selected urls in TakedownStore
# show_selected_urls(data)


## Help with email request
If you would like to make the request via email, input your information below to generate draft email text.

In [ ]:
from CourseZero.Widgets.TakedownInputs import make_text_input

def takedown_area(output, data_store):
    inputs = [make_text_input(r) for r in TakedownStore.input_fields]
    with output:
        display(inputs)

def show_email_draft(output):
    if not TakedownStore.is_ready: pass
    
    with open('{}/takedown_request.html'.format(env.TEMPLATE_FOLDER), 'r') as o:
        template = o.read()
    output.clear_output()
    with output:
        display(widgets.HTML(value=template.format(**TakedownStore.format_args())))